In [199]:
import pickle as pkl
import pandas as pd
import numpy as np
import datetime as dt
import re
import sys

In [200]:
donors = pd.read_pickle('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/df_donor_clean-Copy1.pkl')
c_approval = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_c_approval_Final.csv')
p_approval = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_p_approval_Final.csv')
generic_approval = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_Generic_Final.csv')
direction_approval = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_Direction_Final.csv')

#Copies
minimum_num_donors = donors
congress = c_approval
president = p_approval
generic = generic_approval
direction = direction_approval

In [217]:
####################
# helper functions #
####################

# For the "congressional job approval" category in cpgd_final table, the cleaned approval ratings 
# correspond to approval ratings for the republicans and the disapproval ratings column values
# correspond to democrats. Meaning that both columns in this case only represent
# approval ratings for each party. All other category column values are just as expected. 

flatten = lambda lst: [item for sublist in lst for item in sublist]  


def containsAny(str, set):
    return 1 in [element in str for element in set]


def quantizeSpread(lst):
    disapprovalSet = {'disapprove', 'Disapprove', 'Wrong Direction', 'Wrong Track'}
    tieSet = {'Tie', 'tie'}
    for i in range(len(lst)):
        if containsAny(lst[i], disapprovalSet):
            lst[i] = re.sub(r'[^0-9]+', "-", lst[i])
        elif containsAny(lst[i], tieSet):
            lst[i] = re.sub(r'[^0-9]+', "0", lst[i])
        else:
            lst[i] = re.sub(r'[^0-9]', "", lst[i])     
    if flatten([re.findall(r'[^0-9\-]', i) for i in lst]):
        raise Exception(lst)
    return lst

def approvalTableCleanup(table, approvalType):
    splitColumn = [i.split(',') for i in table['Results']]
    approvalCol = [splitColumn[i][0] for i in range(len(splitColumn))]
    disapprovalCol = [splitColumn[i][1] for i in range(len(splitColumn))]
#     table['approval rating'] = [re.sub(r'[^0-9]', "", i) for i in approvalCol]
    approvalCol = [re.sub(r'[^0-9]', "", i) for i in approvalCol]
    table['approval rating'] = [int(i) for i in approvalCol]
#     table['disapproval rating'] = [re.sub(r'[^0-9]', "", i) for i in disapprovalCol]
    disapprovalCol = [re.sub(r'[^0-9]', "", i) for i in disapprovalCol]
    table['disapproval rating'] = [int(i) for i in disapprovalCol]
    table = table.drop('Results', axis = 1)
    table = table.drop('Poll', axis = 1)
    table = table.drop('Year', axis = 1)
    stringSpreads = table['Spread'].tolist()
    spreadCol = quantizeSpread(stringSpreads)
    table['spread'] = [int(i) for i in spreadCol]
    table = table.drop('Spread', axis = 1)
    table.columns = map(str.lower, table.columns)
    table = table.rename(index = str, columns = {"race/topic (click to sort)": "approval type"})
    table['approval type'] = table['approval type'].str.lower()
    table = table.set_index('date')
    table = table.groupby('date').mean()
    table['approval type'] = [approvalType for i in range(len(table))]
    return table

def stack_cpgd_data(frames):
    agg_table = pd.concat(frames)
    return agg_table

donorLevels = dict([('d1', 200), ('d2', 499.99), ('d3', 999.99), ('d4', 1999.99), ('d5', 2000)])
adHeur = lambda col, level: np.divide(col, donorLevels[level])

def donor_level_features(table):
    
    """Adds features to the FEC table that include the minimum number of possible donors
    per donor level"""
    
    adHeur = lambda col, level: np.divide(col, donorLevels[level])
    donorLevels = dict([('d1', 200), ('d2', 499.99), ('d3', 999.99), ('d4', 1999.99), ('d5', 2000)])
    
    table['estimated_num_d1_donors'] = adHeur(table['Donation Level 1'], 'd1')
    table['estimated_num_d2_donors'] = adHeur(table['Donation Level 2'], 'd2')
    table['estimated_num_d3_donors'] = adHeur(table['Donation Level 3'], 'd3')
    table['estimated_num_d4_donors'] = adHeur(table['Donation Level 4'], 'd4')
    table['estimated_num_d5_donors'] = adHeur(table['Donation Level 5'], 'd5')
    return table


In [202]:
c_final = approvalTableCleanup(congress, 'Congressional Job Approval')
p_final = approvalTableCleanup(president, 'President Obama Job Approval')
g_final = approvalTableCleanup(generic, 'Generic Congressional Vote')
d_final = approvalTableCleanup(direction, 'Direction of Country')

In [203]:
cpgd_final_frames = [c_final, p_final, g_final, d_final]
cpgd_final = stack_cpgd_data(cpgd_final_frames)

In [194]:
cpgd_final.to_pickle('cpgd_actual_final.pkl')

In [218]:
# For the "congressional job approval" category in cpgd_final table, the cleaned approval ratings 
# correspond to approval ratings for the republicans and the disapproval ratings column values
# correspond to democrats. Meaning that both columns in this case only represent
# approval ratings for each party. All other category column values are just as expected. 

cpgd_final




,approval rating,disapproval rating,spread,approval type
date,,,,
2010-04-21,20.000000,71.0,-51.000000,Congressional Job Approval
2010-04-22,22.000000,68.0,-46.000000,Congressional Job Approval
2010-05-12,21.000000,72.0,-51.000000,Congressional Job Approval
2010-05-14,21.000000,75.0,-54.000000,Congressional Job Approval
2010-05-15,28.000000,71.0,-43.000000,Congressional Job Approval
2010-05-21,22.000000,65.0,-43.000000,Congressional Job Approval
2010-05-25,15.000000,77.0,-62.000000,Congressional Job Approval
2010-06-08,26.000000,71.0,-45.000000,Congressional Job Approval
2010-06-11,22.000000,69.0,-47.000000,Congressional Job Approval


In [150]:
# direction of country
# congressional approval
# presidential approval
# generic congressional vote


#Copies
# minimum_num_donors = donors
# congress = c_approval
# president = p_approval
# generic = generic_approval
# direction = direction_approval

In [216]:
# c_approval.head()

In [215]:
# p_approval.head()

In [214]:
# generic.head()

In [213]:
# direction.head()

In [130]:
# people = lambda people: 'franken' in people
# mask = minimum_num_donors.race_candidates.apply(people)
# temp = minimum_num_donors[mask]
# temp

# # def test():
# #     if name != winner:

# temp.columns

In [212]:
# congress.head()